<h1>The Battle of Neighborhoods (Week 1)

<b><u>Introduction and Data Sections

In [2]:
import numpy as np # library to handle data in a vectorized manner
import time
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


<h3>Coursera Capstone - REPORT CONTENT</h3><br>
<b>1-</b> Introduction Section : Discussion of the business problem and the interested audience in this project.<br>
<b>2-</b> Data Section : Description of the data that will be used to solve the problem and the sources.<br>
<b>3-</b> Methodology section : Discussion and description of exploratory data analysis carried out, any inferential statistical testing performed, and if any machine learnings were used establishing the strategy and purposes.<br>
<b>4-</b> Results section : Discussion of the results.<br>
<b>5-</b> Discussion section : Elaboration and discussion on any observations noted and any recommendations suggested based on the results.<br>
<b>6-</b> Conclusion section : Report Conclusion.<br>

<h2>1. Introduction Section :</h2><p>
<h3>Discussion of the business problem and the audience who would be interested in this project.</h3><p>
<h3>Description of the Problem and Background</h3><p>
<b><u>Scenario:</u></b>
I am a data scientist residing in Niterói, Rio de Janeiro, Brazil. I currently live within walking distance to Bus station and I enjoy many ammenities and venues in the area, such as various international cousine restaurants, cafes, food shops and entertainment.

I have been offered a great opportunity to work for a leader firm in Manhattan, NY. I am very excited and I want to use this opportunity to practice my learnings in Coursera in order to answer relevant questions arisen. The key question is : How can I find a convenient and enjoyable place similar to mine now in Niterói, Rio de Janeiro?

Certainly, I can use available real estate apps and Google but the idea is to use and apply myself the learned tools during the course. In order to make a comparison and evaluation of options in Manhattan NY, I must set some basis, therefore the apartment in Manhattan must meet the following demands:

&emsp;&emsp;- desired location is near a bus station in the Manhattan area and within 1.0 mile (1.6 km) radius<br>
&emsp;&emsp;- top ammenities in the selected neighborhood shall be similar to current residence<br>
&emsp;&emsp;- desirable to have venues such as coffee shops, wine stores, gym and food shops<br>
&emsp;&emsp;- as a reference, I have included a map of venues near current residence in Niterói, Rio de Janeiro.<p>
    
<b><u>Business Problem:</u></b>
The challenge is to find a suitable apartment for rent in Icaraí RJ that complies with the demands on location and venues. The data required to resolve this challenge is described in the following section 2, below.

<b><u>Interested Audience:</u></b>
I believe this is a relevant challenge with valid questions for anyone moving to other large city in US, EU or Asia. The same methodology can be applied in accordance to demands as applicable. This case is also applicable for anyone interested in exploring starting or locating a new business in any city. Lastly, it can also serve as a good practical exercise to develop Data Science skills.

<h2>2. Data Section:</h2><p>
<h3>Description of the data and its sources that will be used to solve the problem</h3><p>
<h3>Description of the Data:</h3><p>
<b>The following data is required to answer the issues of the problem:</b><p>

&emsp;&emsp;- List of Boroughs and neighborhoods of Manhattan with their geodata (latitud and longitud)<br>
&emsp;&emsp;- List of Subway metro stations in Manhattan with their address location<br>
&emsp;&emsp;- List of apartments for rent in Manhattan area with their addresses and price<br>
&emsp;&emsp;- Preferably, a list of apartment for rent with additional information, such as price, address, area, # of beds, etc<br>
&emsp;&emsp;- Venues for each Manhattan neighborhood ( than can be clustered)<br>
&emsp;&emsp;- Venues for subway metro stations, as needed<br><p>
<h3>How the data will be used to solve the problem</h3><p>
The data will be used as follows:<p>

&ensp;&ensp;-&nbsp;Use Foursquare and geopy data to map top 10 venues for all Manhattan neighborhoods and clustered in groups ( as per Course LAB)<br>
&ensp;&ensp;-&nbsp;Use foursquare and geopy data to map the location of subway metro stations , separately and on top of the above clustered map in order to be able to identify the venues and ammenities near each metro station, or explore each subway location separately<br>
&ensp;&ensp;-&nbsp;Use Foursquare and geopy data to map the location of rental places, in some form, linked to the subway locations.<br>
&ensp;&ensp;-&nbsp;Create a map that depicts, for instance, the average rental price per square ft, around a radious of 1.0 mile (1.6 km) around each subway station - or a similar metrics. I will be able to quickly point to the popups to know the relative price per subway area.<br>
&ensp;&ensp;-&nbsp;Addresses from rental locations will be converted to geodata( lat, long) using Geopy-distance and Nominatim.<br>
&ensp;&ensp;-&nbsp;Data will be searched in open data sources if available, from real estate sites if open to reading, libraries or other government agencies such as Metro New York MTA, etc.<br><p>

The procesing of these DATA will allow to answer the key questions to make a decision:<p>

&ensp;&ensp;-&nbsp;What is the cost of rent (per square ft) around a mile radius from each subway metro station?<br>
&ensp;&ensp;-&nbsp;What is the area of Manhattan with best rental pricing that meets criteria established?<br>
&ensp;&ensp;-&nbsp;What is the distance from work place ( Park Ave and 53 rd St) and the tentative future home?<br>
&ensp;&ensp;-&nbsp;What are the venues of the two best places to live? How the prices compare?<br>
&ensp;&ensp;-&nbsp;How venues distribute among Manhattan neighborhoods and around metro stations?<br>
&ensp;&ensp;-&nbsp;Are there tradeoffs between size and price and location?<br>
&ensp;&ensp;-&nbsp;Any other interesting statistical data findings of the real estate and overall data.<br><p>

    
<h3>Reference of venues around current residence in Brazil for comparison to Manhattan place</h3>

In [20]:
# Niteroi, Rio de Janeiro, Brazil
address = 'Rua Mem de Sá, Niterói, RJ, Brazil'

geolocator = Nominatim(user_agent="nd_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Niterói home are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Niterói home are -22.9033566, -43.1086639.


In [21]:
neighborhood_latitude=latitude
neighborhood_longitude=longitude

In [22]:
CLIENT_ID = '3J5FMR12CZVFID0LMWXRKRGXR44WG1TOENUAFWGSFLDFKDAI' # your Foursquare ID
CLIENT_SECRET = 'KEJFEMCFUOBULPLCCQKOKTTT43K5E4ZWPP20HEDUBZ2T2HNR' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 3J5FMR12CZVFID0LMWXRKRGXR44WG1TOENUAFWGSFLDFKDAI
CLIENT_SECRET:KEJFEMCFUOBULPLCCQKOKTTT43K5E4ZWPP20HEDUBZ2T2HNR


In [23]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 2500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=3J5FMR12CZVFID0LMWXRKRGXR44WG1TOENUAFWGSFLDFKDAI&client_secret=KEJFEMCFUOBULPLCCQKOKTTT43K5E4ZWPP20HEDUBZ2T2HNR&v=20180604&ll=-22.9033566,-43.1086639&radius=2500&limit=100'

In [24]:
results = requests.get(url).json()
#results

In [25]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [18]:
venues = results['response']['groups'][0]['items']
    
NDnearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
NDnearby_venues =NDnearby_venues.loc[:, filtered_columns]

# filter the category for each row
NDnearby_venues['venue.categories'] = NDnearby_venues.apply(get_category_type, axis=1)

# clean columns
NDnearby_venues.columns = [col.split(".")[-1] for col in NDnearby_venues.columns]

NDnearby_venues.head(10)

,name,categories,lat,lng
0,Campo de São Bento,Park,-22.904364,-43.107562
1,Temakeria e Cia,Japanese Restaurant,-22.904560,-43.110627
2,Confeitaria Beira Mar,Bakery,-22.905444,-43.112172
3,Big Boss Bistrô & Grill,Steakhouse,-22.901284,-43.110953
4,Dice'n'Roll Coffee Tales,Coffee Shop,-22.902703,-43.106526
5,Squasso Centro de beleza e Spa,Salon / Barbershop,-22.905827,-43.109923
6,Porção Mágica,Dessert Shop,-22.903673,-43.110272
7,Kopenhagen,Chocolate Shop,-22.902128,-43.111771
8,Calçadão de Icaraí,Pedestrian Plaza,-22.904246,-43.114734
9,Crema & Cioccolato Café,Ice Cream Shop,-22.909079,-43.107834


<h2>Map of Niterói, Rio de Janeiro - Brazil with venues near residence place - for reference

In [26]:
# create map of New Delhi place  using latitude and longitude values
map_nd = folium.Map(location=[latitude, longitude], zoom_start=14)

# add markers to map
for lat, lng, label in zip(NDnearby_venues['lat'], NDnearby_venues['lng'], NDnearby_venues['name']):
    label = folium.Popup(label, parse_html=True)
    folium.RegularPolygonMarker(
        [lat, lng],
        number_of_sides=4,
        radius=10,
        popup=label,
        color='blue',
        fill_color='#0f0f0f',
        fill_opacity=0.7,
    ).add_to(map_nd)  
    
map_nd